In [40]:
from scipy.sparse import hstack, csr_matrix,vstack
import os
import gensim
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from gensim.models import KeyedVectors
from nltk import WordNetLemmatizer, PorterStemmer, RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
import gensim.downloader as api
import warnings
from transformers import TFDistilBertModel, DistilBertTokenizerFast, DistilBertConfig, BertModel, BertTokenizer
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

import torch
import transformers as ppb # pytorch transformers


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB

import warnings

import swifter
import tqdm


In [2]:
# path = '/content/Insight_Stress_Analysis/data/'
path = '../data/'
train = pd.read_csv(path + 'dreaddit-train-processed.csv', encoding = "ISO-8859-1")
test = pd.read_csv(path + 'dreaddit-test-processed.csv', encoding = "ISO-8859-1")

In [3]:
features_train = train.drop(['subreddit','post_id','sentence_range','text','id','label','confidence','social_timestamp','social_karma','syntax_ari','processed_text'], 1)
features_test = test.drop(['subreddit','post_id','sentence_range','text','id','label','confidence','social_timestamp','social_karma','syntax_ari','processed_text'], 1)

print(features_train.columns)
features_len = len(features_train.columns)

Index(['Unnamed: 0', 'lex_liwc_WC', 'lex_liwc_Analytic', 'lex_liwc_Clout',
       'lex_liwc_Authentic', 'lex_liwc_Tone', 'lex_liwc_WPS',
       'lex_liwc_Sixltr', 'lex_liwc_Dic', 'lex_liwc_function',
       ...
       'lex_dal_min_pleasantness', 'lex_dal_min_activation',
       'lex_dal_min_imagery', 'lex_dal_avg_activation', 'lex_dal_avg_imagery',
       'lex_dal_avg_pleasantness', 'social_upvote_ratio',
       'social_num_comments', 'syntax_fk_grade', 'sentiment'],
      dtype='object', length=107)


In [4]:
def flatten_lists(lst):
    """Remove nested lists."""
    return [item for sublist in lst for item in sublist]

In [5]:
w2v = gensim.models.Word2Vec(list(train["processed_text"]), window=10, min_count=2,
                                vector_size=300,
                                workers=10,
                                epochs=100)
glove = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [6]:
def document_vector(doc):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    doc = [word for word in doc if word in list(w2v.wv.index_to_key)]
    return np.mean(w2v.wv[doc], axis=0)


def document_vector_glove(doc):
    """Create document vectors by averaging word vectors. Remove out-of-vocabulary words."""
    word_vecs = list()
    for word in doc:
        try:
            word_vec = glove.word_vec(word)
        except KeyError:
            word_vec = np.zeros([300], dtype=np.float32)
        word_vecs.append(word_vec)
    return np.mean(word_vecs, axis=0)

## Model 1: Domain Word2vec

In [7]:
train['doc_vector'] = train["processed_text"].apply(document_vector)
test['doc_vector'] = test["processed_text"].apply(document_vector)

In [8]:
train_f = np.stack(train['doc_vector'])
test_f = np.stack(test['doc_vector'])

In [9]:
print(train_f.shape)
print(features_train.shape)

(2838, 300)
(2838, 107)


In [10]:
## Append Features
train_f = np.concatenate((features_train, train_f), axis=1)
test_f = np.concatenate((features_test, test_f), axis=1)
# train_f = hstack([features_train,train_f])
# test_f = hstack([features_test,test_f])

In [11]:
print(train_f.shape)
print(test_f.shape)

(2838, 407)
(715, 407)


In [12]:
X = list(train_f)
y = list(train["label"])
X_test = list(test_f)
y_test = list(test['label'])

In [13]:
clf = LogisticRegression(C=100)

In [14]:
clf.fit(X, y)

LogisticRegression(C=100)

In [15]:
y_pred = clf.predict(X_test)

In [16]:
print(f1_score(y_pred,y_test))

0.7596899224806202


## Model 2: Pretrained Word2vec #TODO change with pubmed word2vec

In [28]:
train['doc_vector'] = train["processed_text"].apply(document_vector_glove)
test['doc_vector'] = test["processed_text"].apply(document_vector_glove)

train_f = np.stack(train['doc_vector'])
test_f = np.stack(test['doc_vector'])

In [29]:
train_f = np.concatenate((features_train, train_f), axis=1)
test_f = np.concatenate((features_test, test_f), axis=1)

In [30]:
X = list(train_f)
y = list(train["label"])
X_test = list(test_f)
y_test = list(test['label'])

In [31]:
clf = LogisticRegression(C=100)

In [32]:
clf.fit(X, y)

LogisticRegression(C=100)

In [33]:
y_pred = clf.predict(X_test)

In [34]:
print(f1_score(y_pred,y_test))

0.761038961038961


## Model 3: Bert word2vec

In [35]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#
bert_model = BertModel.from_pretrained("bert-base-uncased").to("cuda")
# tokenized_text = torch.tensor(tokenized_dict["input_ids"])
# with torch.no_grad():
#   embeddings = bert_model(torch.tensor(tokenized_text.unsqueeze(0)))
#
# get_cls = lambda x: x[0][:, 0, :].squeeze()
# flatten_embed = lambda x: torch.flatten(x[0])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:

class BertTokenizer(object):

    def __init__(self, text=[]):
        self.text = text

        self.model_class, self.tokenizer_class, self.pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
        # Load pretrained model/tokenizer
        self.tokenizer = self.tokenizer_class.from_pretrained(self.pretrained_weights)

        self.model = self.model_class.from_pretrained(self.pretrained_weights).to("cuda")

    def get(self):

        df = pd.DataFrame(data={"text":self.text})
        tokenized = df["text"].swifter.apply((lambda x: self.tokenizer.encode(x, add_special_tokens=True)))

        max_len = 256

        padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

        attention_mask = np.where(padded != 0, 1, 0)
        input_ids = torch.tensor(padded).cuda()
        attention_mask = torch.tensor(attention_mask).cuda()

        with torch.no_grad(): last_hidden_states = self.model(input_ids, attention_mask=attention_mask)
        
        features = last_hidden_states[0][:, 0, :].numpy()

        return features

In [48]:
torch.cuda.empty_cache()

_instance =BertTokenizer(text=train["processed_text"])
tokens_train = _instance.get()
_instance2 = BertTokenizer(text=test["processed_text"])
tokens_test = _instance2.get()

features_tensor_train = torch.tensor(features_train)
features_tensor_test = torch.tensor(features_test)

lr_clf = LogisticRegression(C=100)
lr_clf.fit(torch.cat(tokens_train,features_tensor_train), train["label"])

predicted = lr_clf.predict(torch.cat(tokens_test,features_tensor_test))

print(f1_score(predicted,test["label"]))

KeyboardInterrupt: 